In [3]:
from pathlib import Path
import math
import warnings
import pandas as pd
import numpy as np
import yfinance as yf

pd.set_option("display.float_format", lambda x: f"{x:,.4f}")
warnings.filterwarnings("ignore")


TICKER_STRING = "WRD NCLH SMPL AI"

TICKERS = [t.strip().upper() for t in TICKER_STRING.split() if t.strip()]
print("Tickers:", TICKERS)
"""))

# Helper functions
cells.append(nbf.v4.new_code_cell("""
from typing import Optional, Dict, Any

def safe_first(series: pd.Series, prefer_positive: bool=False) -> Optional[float]:
    if series is None or len(series) == 0:
        return np.nan
    vals = series.dropna().astype(float).values
    if vals.size == 0:
        return np.nan
    if prefer_positive:
        for v in vals:
            if v > 0:
                return float(v)
    return float(vals[0])

def effective_tax_rate(income_stmt: pd.DataFrame) -> float:
    try:
        tax = safe_first(income_stmt.loc["Tax Provision"] if "Tax Provision" in income_stmt.index 
                         else income_stmt.loc["Income Tax Expense"], prefer_positive=False)
        pretax = safe_first(income_stmt.loc["Pretax Income"], prefer_positive=True)
        if pd.isna(tax) or pd.isna(pretax) or pretax == 0:
            return 0.21  # fallback (US corporate baseline-ish)
        rate = max(0.0, min(0.35, float(tax) / float(pretax)))
        return rate
    except Exception:
        return 0.21

def compute_metrics(tick: str) -> Dict[str, Any]:
    t = yf.Ticker(tick)

    # Price & EPS for P/E
    price = np.nan
    trailing_eps = np.nan
    trailing_pe = np.nan

    # Try fast_info (price)
    try:
        if hasattr(t, "fast_info") and t.fast_info is not None:
            price = float(t.fast_info.get("last_price", np.nan))
    except Exception:
        pass

    # Try info dict as a fallback for EPS/PE
    info = {}
    try:
        info = t.get_info() if hasattr(t, "get_info") else (t.info if hasattr(t, "info") else {})
    except Exception:
        info = {}

    if math.isnan(price):
        price = float(info.get("currentPrice", np.nan) or info.get("regularMarketPrice", np.nan) or np.nan)

    trailing_eps = info.get("trailingEps", np.nan)
    trailing_pe = info.get("trailingPE", np.nan)

    # If we don't have trailing PE but have price & EPS, compute
    if (pd.isna(trailing_pe) or trailing_pe in (None,)) and not pd.isna(price) and not pd.isna(trailing_eps) and trailing_eps not in (0, None):
        trailing_pe = price / trailing_eps

    # Financial statements
    income_stmt = t.income_stmt  # yfinance 0.2.40+
    if income_stmt is None or income_stmt.empty:
        income_stmt = t.financials  # yfinance legacy

    balance_sheet = t.balance_sheet if hasattr(t, "balance_sheet") else t.balance_sheet
    if balance_sheet is None or balance_sheet.empty:
        balance_sheet = t.balance_sheet

    cashflow = t.cashflow if hasattr(t, "cashflow") else t.cashflow

    # Normalize column orientation (ensure latest period is first column)
    def normalize(df: Optional[pd.DataFrame]) -> pd.DataFrame:
        if df is None or df.empty:
            return pd.DataFrame()
        # Ensure rows are named consistently
        df = df.copy()
        if not isinstance(df.index, pd.Index):
            df.index = pd.Index(df.index)
        # Sort columns descending by date if they are datetime-like
        try:
            cols = pd.to_datetime(df.columns)
            df = df.loc[:, cols.sort_values(ascending=False).index]
        except Exception:
            pass
        return df

    income_stmt = normalize(income_stmt)
    balance_sheet = normalize(balance_sheet)
    cashflow = normalize(cashflow)

    # --- Gross Profit Margin ---
    revenue = np.nan
    gross_profit = np.nan
    try:
        # Attempt common line names
        if "Total Revenue" in income_stmt.index:
            revenue = safe_first(income_stmt.loc["Total Revenue"])
        elif "Revenues" in income_stmt.index:
            revenue = safe_first(income_stmt.loc["Revenues"])
        elif "Revenue" in income_stmt.index:
            revenue = safe_first(income_stmt.loc["Revenue"])
        if "Gross Profit" in income_stmt.index:
            gross_profit = safe_first(income_stmt.loc["Gross Profit"])
        elif "GrossProfit" in income_stmt.index:
            gross_profit = safe_first(income_stmt.loc["GrossProfit"])
    except Exception:
        pass
    gross_margin = (gross_profit / revenue) if revenue and not pd.isna(revenue) and revenue != 0 else np.nan

    # --- ROIC (approx) ---
    # NOPAT
    ebit = np.nan
    try:
        # Prefer Operating Income; fallback to EBIT if provided
        if "Operating Income" in income_stmt.index:
            ebit = safe_first(income_stmt.loc["Operating Income"], prefer_positive=True)
        elif "EBIT" in income_stmt.index:
            ebit = safe_first(income_stmt.loc["EBIT"], prefer_positive=True)
    except Exception:
        pass

    tax_rate = effective_tax_rate(income_stmt) if not income_stmt.empty else 0.21
    nopat = (ebit * (1.0 - tax_rate)) if not pd.isna(ebit) else np.nan

    # Invested Capital = Total Debt + Total Equity - Cash
    total_debt = np.nan
    try:
        short_debt = 0.0
        long_debt = 0.0
        if "Short Long Term Debt" in balance_sheet.index:
            short_debt = safe_first(balance_sheet.loc["Short Long Term Debt"], prefer_positive=True) or 0.0
        if "Short Term Debt" in balance_sheet.index:
            short_debt = safe_first(balance_sheet.loc["Short Term Debt"], prefer_positive=True) or short_debt
        if "Long Term Debt" in balance_sheet.index:
            long_debt = safe_first(balance_sheet.loc["Long Term Debt"], prefer_positive=True) or 0.0
        total_debt = (short_debt or 0.0) + (long_debt or 0.0)
    except Exception:
        pass

    total_equity = np.nan
    try:
        if "Total Stockholder Equity" in balance_sheet.index:
            total_equity = safe_first(balance_sheet.loc["Total Stockholder Equity"])
        elif "Total Equity Gross Minority Interest" in balance_sheet.index:
            total_equity = safe_first(balance_sheet.loc["Total Equity Gross Minority Interest"])
    except Exception:
        pass

    cash = np.nan
    try:
        for label in ["Cash And Cash Equivalents", "Cash And Cash Equivalents And Short Term Investments", "Cash"]:
            if label in balance_sheet.index:
                cash = safe_first(balance_sheet.loc[label])
                break
    except Exception:
        pass

    invested_capital = np.nan
    try:
        if not pd.isna(total_debt) or not pd.isna(total_equity) or not pd.isna(cash):
            invested_capital = (0.0 if pd.isna(total_debt) else total_debt) + \
                               (0.0 if pd.isna(total_equity) else total_equity) - \
                               (0.0 if pd.isna(cash) else cash)
            if invested_capital == 0:
                invested_capital = np.nan
    except Exception:
        pass

    roic = (nopat / invested_capital) if (not pd.isna(nopat) and not pd.isna(invested_capital) and invested_capital != 0) else np.nan

    # --- Free Cash Flow Margin ---
    operating_cf = np.nan
    capex = np.nan
    try:
        if "Operating Cash Flow" in cashflow.index:
            operating_cf = safe_first(cashflow.loc["Operating Cash Flow"])
        elif "Total Cash From Operating Activities" in cashflow.index:
            operating_cf = safe_first(cashflow.loc["Total Cash From Operating Activities"])
        if "Capital Expenditure" in cashflow.index:
            capex = safe_first(cashflow.loc["Capital Expenditure"])
        elif "Investments In Property Plant And Equipment" in cashflow.index:
            capex = safe_first(cashflow.loc["Investments In Property Plant And Equipment"])
    except Exception:
        pass

    free_cash_flow = np.nan
    try:
        if not pd.isna(operating_cf) and not pd.isna(capex):
            free_cash_flow = float(operating_cf) - float(capex)
    except Exception:
        pass

    fcf_margin = (free_cash_flow / revenue) if (not pd.isna(free_cash_flow) and not pd.isna(revenue) and revenue != 0) else np.nan

    # --- Interest Coverage ---
    # Use EBIT / |Interest Expense|
    interest_expense = np.nan
    try:
        for label in ["Interest Expense", "Interest Expense Non Operating"]:
            if label in income_stmt.index:
                interest_expense = safe_first(income_stmt.loc[label])
                break
    except Exception:
        pass

    interest_coverage = np.nan
    if not pd.isna(ebit) and not pd.isna(interest_expense):
        denom = abs(float(interest_expense)) if float(interest_expense) != 0 else np.nan
        interest_coverage = (float(ebit) / denom) if denom and not pd.isna(denom) else np.nan

    out = {
        "Ticker": tick,
        "Price": price,
        "Revenue": revenue,
        "Gross Profit": gross_profit,
        "Gross Profit Margin": gross_margin,
        "EBIT (Operating Income)": ebit,
        "Effective Tax Rate": tax_rate,
        "NOPAT (approx)": nopat,
        "Total Debt (approx)": total_debt,
        "Total Equity": total_equity,
        "Cash & Equivalents": cash,
        "Invested Capital (approx)": invested_capital,
        "ROIC (approx)": roic,
        "Operating Cash Flow": operating_cf,
        "CapEx": capex,
        "Free Cash Flow": free_cash_flow,
        "Free Cash Flow Margin": fcf_margin,
        "Interest Expense": interest_expense,
        "Interest Coverage": interest_coverage,
        "Trailing EPS": trailing_eps,
        "P/E (trailing)": trailing_pe
    }
    return out

rows = []
for tick in TICKERS:
    try:
        rows.append(compute_metrics(tick))
    except Exception as e:
        rows.append({
            "Ticker": tick,
            "Error": str(e)
        })

df = pd.DataFrame(rows)

# Select & order the clean output columns
clean_cols = [
    "Ticker",
    "Gross Profit Margin",
    "ROIC (approx)",
    "Free Cash Flow Margin",
    "Interest Coverage",
    "P/E (trailing)"
]

# Some users might not get all fields due to data availability; ensure columns exist.
for c in clean_cols:
    if c not in df.columns:
        df[c] = np.nan

clean = df[clean_cols].copy()

# Format as percentages where appropriate
pct_cols = ["Gross Profit Margin", "ROIC (approx)", "Free Cash Flow Margin"]
for c in pct_cols:
    clean[c] = (clean[c] * 100.0).round(2)

# Round ratios
clean["Interest Coverage"] = clean["Interest Coverage"].astype(float).round(2)
clean["P/E (trailing)"] = clean["P/E (trailing)"].astype(float).round(2)

# Pretty column names
clean = clean.rename(columns={
    "Gross Profit Margin": "Gross Margin (>60%)",
    "ROIC (approx)": "ROIC (>11%)",
    "Free Cash Flow Margin": "FCF Margin (>20%)",
    "Interest Coverage": "Interest Coverage (>3x)",
    "P/E (trailing)": "P/E"
})

clean

Tickers: ['WRD', 'NCLH', 'SMPL', 'AI']


,Ticker,Gross Margin (>60%),ROIC (>11%),FCF Margin (>20%),Interest Coverage (>3x),P/E
0,WRD,30.6600,-75.7600,-140.6900,-770.2700,-7.0800
1,NCLH,39.9900,11.2700,34.4000,1.9600,13.7100
2,SMPL,38.4300,8.3100,16.6900,8.4900,19.4400
3,AI,60.6200,-48.1400,-9.8600,NaN,-5.9200
